In [2]:
from square.client import Client
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

True

In [3]:
client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='sandbox')

orders_api = client.orders

### Search orders

In [4]:
# Search orders
body = {
    'location_ids': ['LV2XZS82XY1QP'],
    'query': {
        'filter': {
        }
    },
    'limit': 5,
    'return_entries': True
}

result = orders_api.search_orders(body)

if result.is_success():
    print(result.body)
elif result.is_error():
    print(result.errors)


{'order_entries': [{'order_id': 'FU6BBIb5lgNN4ezJo8YYVGBor0CZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'dk45czlbXWqudbAMaZ6MUQWaXMKZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': '9M4fYq5tBgDKgDsWNPo7aTaUZOVZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'LdHKfiwB6ulvjs8O9TPyMGxiKwLZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}], 'cursor': '6JPwRGoMndm5PVQNLslIjCCRDrkPkXFNmvG7oq5y6yWQisUMAzHEjdQC95kYAViCQk2EteVb9olZ9wtqYfnuCJrZEtSS0a4t5UYhCGntHTbrWghQJS5yvcsqk4eySDAfn7r42AXBxFdDAJt10w9AOw1h0oUH9erUl8i9SBgJUwAj2yRSwBLgMGYv72yfm4qCfX2xTemzEkEHBriwkHbzQvRtFA5YAAYaMr7yNW62aOXS4qy2swKPrg66NndTMknOSyMcGhciMsCAoz1DfHBuUnkhvf6X2wdoq6jIRrK5n2kyBLyHX5ves9Dl9pbYYO'}


### Get list of order id's

In [5]:
orders_dict = result.body
order_entries_list = orders_dict["order_entries"]

print(len(order_entries_list))
order_id_list = []

for x in range(0, len(order_entries_list)):
    order_id = order_entries_list[x]["order_id"]
    order_id_list.append(order_id)

order_id_list


5


['FU6BBIb5lgNN4ezJo8YYVGBor0CZY',
 'TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY',
 'dk45czlbXWqudbAMaZ6MUQWaXMKZY',
 '9M4fYq5tBgDKgDsWNPo7aTaUZOVZY',
 'LdHKfiwB6ulvjs8O9TPyMGxiKwLZY']

### Batch retrieve orders

In [6]:
batch_orders = client.orders.batch_retrieve_orders(
  body = {
    "order_ids": order_id_list
  }
)

if batch_orders.is_success():
  print(batch_orders.body)
elif batch_orders.is_error():
  print(batch_orders.errors)

{'orders': [{'id': 'FU6BBIb5lgNN4ezJo8YYVGBor0CZY', 'location_id': 'LV2XZS82XY1QP', 'line_items': [{'uid': 'mkzqh3WLSJkpmjF3jiMaJ', 'catalog_object_id': 'QU2JOEL4HG64KCTGLUSLFUK6', 'catalog_version': 1694225358547, 'quantity': '5', 'name': 'Tea', 'variation_name': 'Honey Lemon', 'base_price_money': {'amount': 200, 'currency': 'USD'}, 'gross_sales_money': {'amount': 1000, 'currency': 'USD'}, 'total_tax_money': {'amount': 100, 'currency': 'USD'}, 'total_discount_money': {'amount': 0, 'currency': 'USD'}, 'total_money': {'amount': 1100, 'currency': 'USD'}, 'variation_total_price_money': {'amount': 1000, 'currency': 'USD'}, 'applied_taxes': [{'uid': 'lCw1SLSfOwyj56wWUKXaYB', 'tax_uid': 'sales_tax', 'applied_money': {'amount': 100, 'currency': 'USD'}}], 'item_type': 'ITEM', 'total_service_charge_money': {'amount': 0, 'currency': 'USD'}}], 'taxes': [{'uid': 'sales_tax', 'catalog_object_id': 'G6WSJ5RQDYVBKO5Z6PQH3WJJ', 'catalog_version': 1694225358547, 'name': 'Sales tax', 'percentage': '10.0'

### List of orders

In [7]:
batch_orders_dict = batch_orders.body
batch_orders_list = batch_orders_dict['orders']

### Orders dataframe

In [29]:
# first_order = batch_orders_list[0]
order_list = []

for order in batch_orders_list:
    id = order["id"]
    location_id = order["location_id"]
    created_at = order["created_at"]
    updated_at = order["updated_at"]
    state = order["state"]
    version = order["version"]
    net_amounts = order["net_amounts"]
    customer_id = order["customer_id"]
    line_items = order["line_items"]

    cat_obj_id_list = []
    for item in line_items:
        # print(item)
        cat_obj_id_list.append(item["catalog_object_id"])

    order_list.append((id, location_id, created_at, updated_at, state, version, net_amounts, 
                       customer_id, cat_obj_id_list))

# create DataFrame using data
order_df = pd.DataFrame(order_list, columns =["id", "location_id", "created_at", "updated_at", 
                                              "state", "version", "net_amounts", "customer_id", "cat_obj_id"])

order_df = order_df.explode("cat_obj_id")
order_df.head()

,id,location_id,created_at,updated_at,state,version,net_amounts,customer_id,cat_obj_id
0,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,LV2XZS82XY1QP,2023-09-09T02:09:50.182Z,2023-09-09T02:09:50.182Z,OPEN,1,"{'total_money': {'amount': 1100, 'currency': '...",VK09BGAVBNB1T5F4530K163W5M,QU2JOEL4HG64KCTGLUSLFUK6
1,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,LV2XZS82XY1QP,2023-09-09T02:09:49.475Z,2023-09-09T02:09:49.475Z,OPEN,1,"{'total_money': {'amount': 964, 'currency': 'U...",1P0ZY96849H446231JNDH0V97R,M5XGQR3VGKXQRLU5PCI3KH3C
2,dk45czlbXWqudbAMaZ6MUQWaXMKZY,LV2XZS82XY1QP,2023-09-09T02:09:48.278Z,2023-09-09T02:09:48.278Z,OPEN,1,"{'total_money': {'amount': 657, 'currency': 'U...",6WH79TC7KP6X5AYXWQWDR377TW,G3USA7KKSPWD7ZZ2I2RMQIYL
3,9M4fYq5tBgDKgDsWNPo7aTaUZOVZY,LV2XZS82XY1QP,2023-09-09T02:09:47.722Z,2023-09-09T02:09:47.722Z,OPEN,1,"{'total_money': {'amount': 394, 'currency': 'U...",MXSQ3Q1QE8M80HDTVHHJEZRF50,FLXOYUKIRKSDWCSER6VW7EQZ
4,LdHKfiwB6ulvjs8O9TPyMGxiKwLZY,LV2XZS82XY1QP,2023-09-09T02:09:47.158Z,2023-09-09T02:09:47.158Z,OPEN,1,"{'total_money': {'amount': 175, 'currency': 'U...",T7EM1V4A8J23ZGYNGATK07PNKR,FTJBGLEJPVXCOYXEAXKQSKGO


### Order x Location relationship

In [46]:
order_x_location_df = order_df[["id", "location_id"]]
order_x_location_df = order_x_location_df.drop_duplicates()
order_x_location_df

,id,location_id
0,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,LV2XZS82XY1QP
1,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,LV2XZS82XY1QP
2,dk45czlbXWqudbAMaZ6MUQWaXMKZY,LV2XZS82XY1QP
3,9M4fYq5tBgDKgDsWNPo7aTaUZOVZY,LV2XZS82XY1QP
4,LdHKfiwB6ulvjs8O9TPyMGxiKwLZY,LV2XZS82XY1QP


### Order x Customer relationship

In [36]:
order_x_cust_df = order_df[["id", "customer_id"]]
order_x_cust_df = order_x_cust_df.drop_duplicates()
order_x_cust_df

,id,customer_id
0,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,VK09BGAVBNB1T5F4530K163W5M
1,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,1P0ZY96849H446231JNDH0V97R
2,dk45czlbXWqudbAMaZ6MUQWaXMKZY,6WH79TC7KP6X5AYXWQWDR377TW
3,9M4fYq5tBgDKgDsWNPo7aTaUZOVZY,MXSQ3Q1QE8M80HDTVHHJEZRF50
4,LdHKfiwB6ulvjs8O9TPyMGxiKwLZY,T7EM1V4A8J23ZGYNGATK07PNKR


### Order x Category relationship

In [35]:
order_x_cat_obj_df = order_df[["id", "cat_obj_id"]]
order_x_cat_obj_df = order_x_cat_obj_df.drop_duplicates()
order_x_cat_obj_df

,id,cat_obj_id
0,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,QU2JOEL4HG64KCTGLUSLFUK6
1,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,M5XGQR3VGKXQRLU5PCI3KH3C
2,dk45czlbXWqudbAMaZ6MUQWaXMKZY,G3USA7KKSPWD7ZZ2I2RMQIYL
3,9M4fYq5tBgDKgDsWNPo7aTaUZOVZY,FLXOYUKIRKSDWCSER6VW7EQZ
4,LdHKfiwB6ulvjs8O9TPyMGxiKwLZY,FTJBGLEJPVXCOYXEAXKQSKGO
